In [ ]:
# default_exp core

# Simple Neo4j HTTP API Client

> Simple http client to connect neo4j server

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import os,json,base64
import requests
import pandas as pd
from neo4jtools.dwpc import make_dwpc_query
from neo4jtools.utils import graph_renderer, row_renderer

class SimpleNeo4jHTTPAPIClient:
    
    def __init__(self, url, db='neo4j', userid=None, passwd=None):
        self.url=url
        self.set_serverinfo(url)
        self.setdb(db)
        self.authtoken=None
        if userid is not None:
            self.authtoken=self.get_authtoken(userid, passwd)
        
    def set_serverinfo(self,url):
        resp=requests.get(url)
        obj=resp.json()
        self.bolt_routing=obj['bolt_routing']
        self.transaction=obj['transaction']
        self.bolt_direct=obj['bolt_direct']
        self.neo4j_version=obj['neo4j_version']
        self.neo4j_edition=obj['neo4j_edition']

    def setdb(self, db):
        self.db=db
    
    def execute_read_query(self, query, output_format=['row','graph']):
        return self.execute_query(query, output_format)
    
    def execute_query(self, query, output_format=['row','graph']):
        url=self.transaction.format(databaseName=self.db) + '/commit'
        headers={
            "content-type": "application/json"
        }
        if self.authtoken is not None:
            headers['authorization']="Basic {}".format(self.authtoken)

        if isinstance(output_format, list):
            resultDataContents = output_format
        else:
            resultDataContents = [output_format]

        statement={
          "statements": [
            {
              "statement": query,
              "resultDataContents": resultDataContents
            }
          ]
        }
        resp=requests.post(url, 
                           data=json.dumps(statement), 
                           headers=headers)

        output= resp.json()

        if len(output['errors']) > 0: 
            raise Exception(output['errors'])
            
        
        output_renderer=None
        if output_format=='row':
            output_renderer=row_renderer
                    
        if output_format=='graph':
            output_renderer=graph_renderer
        
        if output_renderer:
            output=output_renderer(output)
        
        return output
    
    
    def calculate_dwpc(self, genes, reltype, hops, 
                       dwpc_score_prop_name='dwpc_score', 
                       only_relations_with_pmid=True,
                       debug=False):
        qry=make_dwpc_query(genes, 
                            reltype=reltype, 
                            hops=hops, 
                            dwpc_score_prop_name=dwpc_score_prop_name, 
                            only_relations_with_pmid=only_relations_with_pmid)
        if debug:
            print(qry)
            
        results=self.execute_read_query(qry, output_format='row')
        
        return results
    
    def __repr__(self):
        return json.dumps({
            'classname':self.__class__.__name__,
            'url':self.url,
            'neo4j_version':self.neo4j_version,
            'neo4j_edition':self.neo4j_edition,
            'db':self.db,
            'auth': self.authtoken is not None
        })

    @staticmethod
    def get_authtoken(userid, passwd):
        authstr='{}:{}'.format(userid, passwd)
        b64token=base64.b64encode(authstr.encode())
        strtoken=b64token.decode()
        return strtoken

## Simple example

In [ ]:
# Connect to a neo4j server without auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.5", "neo4j_edition": "community", "db": "neo4j", "auth": false}

In [ ]:
try:
    client.execute_query('match (n) return count(n);', output_format=['row'])
except Exception as e:
    assert e

In [ ]:
# Connect to a neo4j server with auth info
client=SimpleNeo4jHTTPAPIClient(url='http://localhost:7474', userid='neo4j',passwd='test')
client

{"classname": "SimpleNeo4jHTTPAPIClient", "url": "http://localhost:7474", "neo4j_version": "4.4.5", "neo4j_edition": "community", "db": "neo4j", "auth": true}

In [ ]:
result=client.execute_query('match (n) return count(n);')
result

{'results': [{'columns': ['count(n)'],
   'data': [{'row': [82523],
     'meta': [None],
     'graph': {'nodes': [], 'relationships': []}}]}],
 'errors': []}

## `row_renderer` (output_format='row') example

In [ ]:
result=client.execute_query('match p=(n)-[r]-(m) return n.identifier, n.name, type(r), m.identifier, m.name limit 3;', output_format='row')
result

[{'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '92667',
  'm.name': 'MGME1'},
 {'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '291',
  'm.name': 'SLC25A4'},
 {'n.identifier': 'GO:0000002',
  'n.name': 'mitochondrial genome maintenance',
  'type(r)': 'PARTICIPATES_GpBP',
  'm.identifier': '4205',
  'm.name': 'MEF2A'}]

In [ ]:
pd.DataFrame(result)

,n.identifier,n.name,type(r),m.identifier,m.name
0,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,92667,MGME1
1,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,291,SLC25A4
2,GO:0000002,mitochondrial genome maintenance,PARTICIPATES_GpBP,4205,MEF2A


## `graph_renderer` (output_format='graph') example

In [ ]:
result=client.execute_query('match p=()--() return p limit 3;', output_format='graph')
result

{'nodes': [{'identifier': '92667',
   'name': 'MGME1',
   'label': 'Gene',
   'ensembl_id': 'ENSG00000125871',
   'synonyms': 'C20orf72|DDK1|MTDPS11|bA504H3.4',
   'druggability_source': 'TCRD:v6.12.4',
   'idg_eligible': '0.0',
   'url3': 'https://genetics.opentargets.org/gene/ENSG00000125871',
   'chromosome': '20',
   'url2': 'https://www.genecards.org/cgi-bin/carddisp.pl?gene=MGME1',
   'license': 'CCO 1.0',
   'url': 'http://identifiers.org/ncbigene/92667',
   'pdb': '5ZYV;5ZYW;5ZYU;5ZYT',
   'description': 'mitochondrial genome maintenance exonuclease 1',
   'is_TF': 'False',
   'uniprot_id': 'Q9BQP7;Q5QPE7',
   'druggability_level': 'Tbio',
   'source': 'Entrez Gene:20220425',
   'location': 'Mitochondrion(uniprot);Mitochondria(HPA_main)',
   'xrefs': 'MIM:615076|HGNC:HGNC:16205|Ensembl:ENSG00000125871|AllianceGenome:HGNC:16205',
   'pubtator_score': nan,
   'druggable_url': nan,
   'druggable_sources': nan,
   'is_druggable': nan},
  {'identifier': 'GO:0000002',
   'name': 'mit

In [ ]:
assert 'nodes' in result and 'edges' in result

In [ ]:
pd.DataFrame(result['nodes'])

,identifier,name,label,ensembl_id,synonyms,druggability_source,idg_eligible,url3,chromosome,url2,...,is_TF,uniprot_id,druggability_level,source,location,xrefs,pubtator_score,druggable_url,druggable_sources,is_druggable
0,92667,MGME1,Gene,ENSG00000125871,C20orf72|DDK1|MTDPS11|bA504H3.4,TCRD:v6.12.4,0.0,https://genetics.opentargets.org/gene/ENSG0000...,20,https://www.genecards.org/cgi-bin/carddisp.pl?...,...,False,Q9BQP7;Q5QPE7,Tbio,Entrez Gene:20220425,Mitochondrion(uniprot);Mitochondria(HPA_main),MIM:615076|HGNC:HGNC:16205|Ensembl:ENSG0000012...,NaN,NaN,NaN,NaN
1,GO:0000002,mitochondrial genome maintenance,BiologicalProcess,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Gene Ontology:2021-02-01,NaN,NaN,NaN,NaN,NaN,NaN
2,291,SLC25A4,Gene,ENSG00000151729,AAC1|ANT|ANT 1|ANT1|MTDPS12|MTDPS12A|PEO2|PEO3...,TCRD:v6.12.4;TCDA,0.0,https://genetics.opentargets.org/gene/ENSG0000...,4,https://www.genecards.org/cgi-bin/carddisp.pl?...,...,False,A0A0S2Z3H3;P12235,Tbio,Entrez Gene:20220425,Mitochondrion inner membrane(uniprot);Mitochon...,MIM:103220|HGNC:HGNC:10990|Ensembl:ENSG0000015...,112.281869,https://www.dgidb.org/genes/SLC25A4,"DGIdb:v4.2.0(RussLampel,HopkinsGroom)",True
3,4205,MEF2A,Gene,ENSG00000068305,ADCAD1|RSRFC4|RSRFC9|mef2,TCRD:v6.12.4,0.0,https://genetics.opentargets.org/gene/ENSG0000...,15,https://www.genecards.org/cgi-bin/carddisp.pl?...,...,True,A0A0S2Z454;Q02078;A0A0S2Z4N0;A0A0S2Z4C8;A0A0S2...,Tbio,Entrez Gene:20220425,Nucleus(uniprot);Nucleoplasm(HPA_main),MIM:600660|HGNC:HGNC:6993|Ensembl:ENSG00000068...,NaN,NaN,NaN,NaN


In [ ]:
pd.DataFrame(result['edges'])

,type,start_identifier,start_name,end_identifier,end_name,license,unbiased,is_directional,source
0,PARTICIPATES_GpBP,92667,MGME1,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,True,NCBI gene2go:2021-02-01
1,PARTICIPATES_GpBP,291,SLC25A4,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,True,NCBI gene2go:2021-02-01
2,PARTICIPATES_GpBP,4205,MEF2A,GO:0000002,mitochondrial genome maintenance,CC By 4.0,False,True,NCBI gene2go:2021-02-01
